# Executable Notebook

This notebook trains a number of models on the NOAA active sunspot region data.

There is functionality at the end of the notebook to read sunspot region data from a standard docx file and produce predictions.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from docx2python import docx2python
from imblearn.over_sampling import SMOTE

In [2]:
#Import data

raw_df = pd.read_csv('SRS_all_2001-2020_2.csv')
raw_flares = pd.read_csv('flare_list_2.csv')


In [3]:
#Drop duplicate flares. This sorts the flares by class alphabetically and keeps the last value, so if both classes of flare occur the X occurence is retained.

raw_flares.sort_values('goes_class_ind')
raw_flares.drop_duplicates(subset=['ID'], inplace=True, keep='last')
raw_flares.reset_index(inplace=True)

In [4]:
#This merges the sunspot dataframe with the flare dataframe, matching on the ID (formed in Excel) representing date and location.

merged_df = pd.merge(raw_df[['ID','Area','Z','Mag Type','Number of Sunspots','date']], raw_flares[['ID','goes_class_ind']], 
                     on = 'ID', 
                     how='left')

#Replace any N/As with 0

merged_df = merged_df.fillna(0)

In [5]:
#Split McIntosh parameter into its constituent parts
merged_df['Z-value'] = merged_df['Z'].str[0]
merged_df['p-value'] = merged_df['Z'].str[1]
merged_df['c-value'] = merged_df['Z'].str[2]

#Create binary flare and class columns
merged_df['X_Class'] = [1 if x == 'X' else 0 for x in merged_df['goes_class_ind']]
merged_df['Binary_Flare'] = [1 if x == 'M' or x == 'X' else 0 for x in merged_df['goes_class_ind']]

#Rename and drop columns as appropriate
merged_df.rename(columns={'Number of Sunspots':'Number_of_Sunspots'}, inplace=True)
merged_df.rename(columns={'Mag Type':'Mag_Type'}, inplace=True)

merged_df.drop('Z', axis=1, inplace=True)
merged_df.drop('goes_class_ind', axis=1, inplace=True)


In [6]:
#We need to remove all non-occurences where a flare without a location has occurred in the following 24 hours
#We can use the raw flares list, comparing the full ID to the last 5 digits - all should have the full number of digits as the SRS data does.

merged_df['ID_str'] = merged_df['ID'].astype(str)
merged_df['ID_str'] = merged_df['ID_str'].str[-5:]
merged_df['ID_str'] = merged_df['ID_str'].astype(int)

raw_flares = raw_flares.fillna(0)
raw_flares['ID2'] = raw_flares['ID'].astype(int)

#Merge lists based on this new ID, which matches every event with a flare on that day if there was one
merged_df = pd.merge(merged_df, raw_flares[['ID2']], 
                     left_on = 'ID_str',
                     right_on = 'ID2', 
                     how='left')

#Drop all non-flares that now have an entry in the new ID column (i.e. they have an unlocated flare on the same day)
merged_df = merged_df.drop(merged_df[(merged_df.Binary_Flare == 0) & (merged_df.ID2 > 0)].index)

#Finally, remove temporary ID columns
merged_df.drop('ID_str', axis=1, inplace=True)
merged_df.drop('ID2', axis=1, inplace=True)

In [7]:
#Define dictionaries for encoding

Z_values = {'A' : 0.1, 'H' : 0.15, 'B' : 0.3, 'C' : 0.45, 'D' : 0.6, 'E' : 0.75, 'F' : 0.9}
p_values = {'x' : 0, 'r' : 0.1, 's' : 0.3, 'a' : 0.5, 'h' : 0.7, 'k' : 0.9}
c_values = {'x' : 0, 'o' : 0.1, 'i' : 0.5, 'c' : 0.9}
Mag_values = {'Alpha' : 0, 'Beta' : 0.1, 'Gamma' : 0.5, 'Gamma-Delta' : 0.5, 'Beta-Gamma' : 0.3, 'Beta-Delta' : 0.6, 'Beta-Gamma-Delta' : 0.9,
                'Beta-Gamma.' : 0.3, 'Beta-gamma' : 0.3, 'Beta-delta' : 0.3}

#Map dataframe dictionary values

merged_df['Z-value'] = merged_df['Z-value'].map(Z_values)
merged_df['c-value'] = merged_df['c-value'].map(c_values)
merged_df['p-value'] = merged_df['p-value'].map(p_values)
merged_df['Mag_Type'] = merged_df['Mag_Type'].map(Mag_values)
                

In [9]:
#Create copy of dataframe and remove any columns/features that we don't want to include. Uncomment column headings (after the first three) to remove the named features.

cleaned_df = merged_df.copy()

cleaned_df.pop('ID')
cleaned_df.pop('X_Class')
cleaned_df.pop('date')

#cleaned_df.pop('Mag_Type')
#cleaned_df.pop('Number_of_Sunspots')
#cleaned_df.pop('Area')
#cleaned_df.pop('Z-value')
#cleaned_df.pop('p-value')
#cleaned_df.pop('c-value')

0        01/01/2001
1        01/01/2001
2        01/01/2001
3        01/01/2001
4        01/01/2001
            ...    
24534    29/12/2020
24535    30/12/2020
24536    30/12/2020
24537    31/12/2020
24538    31/12/2020
Name: date, Length: 23461, dtype: object

In [10]:
#Create feature and label vectors for training algorithms.

train_df = cleaned_df
scaler = StandardScaler()
train_labels = np.array(train_df.pop('Binary_Flare'))
train_features = np.array(train_df)
train_features = scaler.fit_transform(train_features)


# OPTIONAL:

Run the following code block to over-sample the minority class. Set rs_count to the number you wish to multiply the minority class by. For example, with rs_count = 6, we are multiplying the number of flaring datapoints by 6.

In [11]:
rs_count = 6

Neg_resample = np.bincount(train_labels)[0]
Pos_resample = rs_count*np.bincount(train_labels)[1]

Class_rebalance = {0:Neg_resample, 1:Pos_resample}

oversample = SMOTE(sampling_strategy=Class_rebalance)

train_features, train_labels = oversample.fit_resample(train_features, train_labels)


# CONTINUE:

Whether re-sampled or not, code may be identically continued from this point onwards.

In [ ]:
#Define models and fit to training data. Any hyperparameter changes must be implemented here. Max_iter may need to be increased if a model does not converge.

MLP_Model = MLPClassifier()                           
RF_Model = RandomForestClassifier()                    
Log_Model = LogisticRegression()

MLP_Model.fit(train_features, train_labels)
RF_Model.fit(train_features, train_labels)
Log_Model.fit(train_features, train_labels)

In [13]:
#Define and compile TensorFlow deep learning model. Input shape must be number of features used. Hyperparameter changes for this model should be implemented here.

model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(6,)),
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
              )

In [14]:
#Fit TensorFlow model to training data. Number of epochs can be altered based on performnce.

model.fit(train_features, train_labels, epochs=20, verbose=0)

#Create probabilistic predictions for TensorFlow model.

probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

# Making a prediction:

By this point we have trained the models on a large set of NOAA data. The code has been designed to make a prediction based on a single docx file. If the formatting of the docx file is changed, the code will not run.

In [15]:
#The docx file must be saved in the current working directory. Ensure that the '.docx' extension is used. One has been left in as an example.
#This creates a dataframe with each row representing an active sunspot region.

SRS_df = pd.DataFrame(docx2python('Space_Weather_Sunspot_Region_Summary_0200_2016-01-02T03-02-06Z.docx').body[1])
SRS_df = SRS_df.iloc[1:,:]
SRS_df = SRS_df.iloc[:-2,:]


In [16]:
#Rename columns and drop unnecessary ones
SRS_df.columns = ['No', 'Loc', 'Lo', 'Area', 'Z', 'LL', 'NN', 'Mag_Type', 'Growth', 'M', 'X', 'P']

SRS_df.drop('Loc', axis=1, inplace=True)
SRS_df.drop('Lo', axis=1, inplace=True)
SRS_df.drop('LL', axis=1, inplace=True)
SRS_df.drop('Growth', axis=1, inplace=True)
SRS_df.drop('M', axis=1, inplace=True)
SRS_df.drop('X', axis=1, inplace=True)
SRS_df.drop('P', axis=1, inplace=True)

#We can see that each entry is formatted as a list. This can be easily rectified by mapping a function to the dataframe.
def unlist(x):
    if isinstance(x, list):
        return x[0]
    else:
        return x

SRS_df = SRS_df.applymap(unlist)


In [17]:
#Encode columns as before.

#Split Z parameter
SRS_df['Z-value'] = SRS_df['Z'].str[0]
SRS_df['p-value'] = SRS_df['Z'].str[1]
SRS_df['c-value'] = SRS_df['Z'].str[2]

#Encode
SRS_df['Z-value'] = SRS_df['Z-value'].map(Z_values)
SRS_df['c-value'] = SRS_df['c-value'].map(c_values)
SRS_df['p-value'] = SRS_df['p-value'].map(p_values)
SRS_df['Mag_Type'] = SRS_df['Mag_Type'].map(Mag_values)

SRS_df = SRS_df.dropna()

In [18]:
#Now fix some formatting issues with non-numerical characters. 

# For all docx files tested with, this was enough to remove any non-numerical characters, however if there is a non-numerical character that is not fixed here that is present
# in the file being tested, then the code will not run.

SRS_df = SRS_df[SRS_df['Area'] != '?']
SRS_df = SRS_df[SRS_df['Area'] != '-']
SRS_df['Area'] = SRS_df['Area'].map(lambda x: x.lstrip('~>').rstrip('*'))

SRS_df['Area'] = SRS_df['Area'].astype(float)
SRS_df['NN'] = SRS_df['NN'].astype(float)

In [19]:
#Create cleaned dataframe copy. If not all features have been used for training, they will need to be removed here.

cleaned_SRS = SRS_df[['Area', 'Mag_Type', 'NN','Z-value', 'p-value', 'c-value']]

#Scale features

SRS_features = np.array(cleaned_SRS)
SRS_features = scaler.transform(SRS_features)

In [20]:
#Make predictions for each model.

MLP_Prediction = MLP_Model.predict_proba(SRS_features)
RF_Prediction = RF_Model.predict_proba(SRS_features)
LogisticReg_Prediction = Log_Model.predict_proba(SRS_features)
TensorFlow_Prediction = probability_model.predict(SRS_features)

# The predictions

Finally, we may produce predicted probabilities. I have just used the MLP and Logistic Regression here as I believe these to be our best two models. With slight adjustment you may also view the random forest and deep learning predictions. They have been set to show probabilities to three decimal places, but again this may be adjusted as suits.


In [31]:

for sunspot in range(len(MLP_Prediction)):
    print('Active Sunspot Region {} Predicted Flaring Probabilities (MLP and LR): {:.3%}, {:.3%}\n'.format(sunspot+1, MLP_Prediction[sunspot][1], LogisticReg_Prediction[sunspot][1]))

Active Sunspot Region 1 Predicted Flaring Probabilities (MLP and LR): 6.172%, 10.392%

Active Sunspot Region 2 Predicted Flaring Probabilities (MLP and LR): 0.272%, 4.015%

Active Sunspot Region 3 Predicted Flaring Probabilities (MLP and LR): 15.728%, 6.005%

Active Sunspot Region 4 Predicted Flaring Probabilities (MLP and LR): 0.646%, 1.610%

